In [1]:
import warnings
warnings.filterwarnings('ignore')

예측 변수 Segmentation, test.csv에 대해 ID별로 Segmentation의 클래스를 예측해서 저장후 제출, 제출 데이터 컬럼은 ID와 Segmentation 두개만 존재해야함. 평가지표는 macro f1 score

In [2]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e4_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e4_p2_test.csv')

display(train.head(2))
test.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,464357,Male,No,40,Yes,Artist,7.0,Low,1.0,Cat_6,A
1,459624,Male,No,18,No,Healthcare,NaN,Low,5.0,Cat_4,D


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,460406,Male,Yes,36,Yes,Healthcare,3.0,Low,2.0,Cat_6
1,466890,Male,Yes,47,Yes,Artist,0.0,Average,6.0,Cat_7


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4881 entries, 0 to 4880
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               4881 non-null   int64  
 1   Gender           4881 non-null   object 
 2   Ever_Married     4785 non-null   object 
 3   Age              4881 non-null   int64  
 4   Graduated        4830 non-null   object 
 5   Profession       4817 non-null   object 
 6   Work_Experience  4368 non-null   float64
 7   Spending_Score   4881 non-null   object 
 8   Family_Size      4689 non-null   float64
 9   Var_1            4831 non-null   object 
 10  Segmentation     4881 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 419.6+ KB


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

In [5]:
# Ever_Married 랑 Graduated 랑 Profession랑 Work_Experience랑 Family_Size랑 
# Var_1에 결측치 존재 근데 다 object

In [6]:
train.Ever_Married.fillna(train.Ever_Married.value_counts().index[0], inplace = True)
train.Graduated.fillna(train.Graduated.value_counts().index[0], inplace = True)
train.Profession.fillna(train.Profession.value_counts().index[0], inplace = True)
train.Var_1.fillna(train.Var_1.value_counts().index[0], inplace = True)
train.Work_Experience.fillna(train.Work_Experience.median(), inplace = True)
train.Family_Size.fillna(train.Family_Size.value_counts().index[0], inplace = True)


# Family_Size 범주형 str로 변경
train.Family_Size = train.Family_Size.astype('str')
test.Family_Size = test.Family_Size.astype('str')

# ID 칼럼 제거

train_drop = train.drop(columns = ['ID', 'Segmentation'])
train_y = train.Segmentation
IDS = test['ID']

test_drop = test.drop(columns = ['ID'])

train_dum = pd.get_dummies(train_drop)
test_dum = pd.get_dummies(test_drop)[train_dum.columns]

#validation - test 구분

x_train, x_test, y_train, y_test = train_test_split(train_dum, train_y, random_state = 42, stratify = train_y)

In [48]:
rf = RandomForestClassifier(n_estimators = 50, max_depth = 12, min_samples_split = 2, min_samples_leaf = 4, random_state = 23, max_features='auto')
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=12, min_samples_leaf=4, n_estimators=50,
                       random_state=23)

In [49]:
pred = rf.predict(x_test)
f1_score(y_test, pred, average = 'macro')

0.4586443722416949

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],  # 트리의 개수
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],  # 리프 노드에 필요한 최소 샘플 수
    'max_features': ['auto', 'sqrt', 'log2'],  # 각 노드에서 사용할 특성의 최대 수
    'bootstrap': [True, False]  # 각 트리가 훈련할 때 샘플을 복원 추출할지 여부
}

In [10]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1')

In [11]:
# grid_search.fit(x_train, y_train)